In [1]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import model_selection, naive_bayes, svm
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, confusion_matrix
from nltk.tokenize import word_tokenize
import pandas as pd
import numpy as np

In [2]:
df_preprocessed = pd.read_csv('C:\\Users\\Janek\\Downloads\\df_train_preprocessed.csv')
df_preprocessed

,Unnamed: 0,File,Text,Category,language
0,0,bb995ead-f65c-43e8-9d7f-771546d21651.tiff,"['parliament', 'wsrdd', 'scrambeer', 'engeyi',...",advertisement,en
1,1,2a476c78-ce1c-4535-831c-626635b69fef.tiff,"['gilvvmather', 'insertion', 'order', 'i6ken7n...",advertisement,en
2,2,7bbd6936-4f3d-4eb4-bb75-983cbca08f48.tiff,"['newport', '902', 'cosmopolitan', 'april', '1...",advertisement,en
3,3,62d852b8-25eb-48e0-8cfe-0091c4f2453e.tiff,"['goforthegołd', 'introduce', 'true', 'gqld', ...",advertisement,en
4,4,8c5a5c2b-c58d-4f7e-985d-236bcc69011e.tiff,"['what', 's', 'tobacoo', 'induisty', 'dong', '...",advertisement,en
...,...,...,...,...,...
10824,10824,3827a69b-52cb-4600-a970-5163322ac4d7.jpg,"['umowa', '0', 'dzielo', 'kukulowo', '0625', '...",umowa_o_dzielo,pl
10825,10825,671cca7a-fe43-4743-9c23-4c4a4ef5b810.jpg,"['umowa', 'odiflo', 'zawrzeć', 'dzień', '20221...",umowa_o_dzielo,pl
10826,10826,a09e8f97-35c0-4327-9ab8-dfc2b2052722.jpg,"['uowaodiflo', 'zawrzeć', 'dzień', '2020032', ...",umowa_o_dzielo,pl
10827,10827,42ede5e5-8b9c-48d7-8996-ca81e8408b90.jpg,"['umowa', '0', 'dziłlo', 'zawrzeć', 'dzień', '...",umowa_o_dzielo,pl


In [3]:
from sklearn.preprocessing import LabelEncoder
Encoder = LabelEncoder()
df_preprocessed['Category'] = Encoder.fit_transform(df_preprocessed['Category'])

In [5]:
# Assuming you have a DataFrame called 'df' with a 'language' column

# Create 'df_pl' by selecting rows where 'language' is 'pl'
df_pl = df_preprocessed[df_preprocessed['language'] == 'pl'].copy()

# Create 'df_en' by selecting rows where 'language' is not 'pl'
df_en = df_preprocessed[df_preprocessed['language'] != 'pl'].copy()

# Reset the index of the resulting DataFrames
df_pl.reset_index(drop=True, inplace=True)
df_en.reset_index(drop=True, inplace=True)

In [17]:
from sklearn.model_selection import train_test_split
df_train, df_en_val = train_test_split(df_en, test_size=0.2, random_state=42)

In [19]:
X_en_train = df_en_train['Text']
y_en_train = df_en_train['Category']
X_en_val = df_en_val['Text']
y_en_val = df_en_val['Category']
X_en_train_tokens= [str(word_tokenize(entry)) for entry in X_en_train]
Tfidf_vect = TfidfVectorizer(max_features=5000)
Tfidf_vect.fit(X_en_train_tokens)
Train_X_Tfidf = Tfidf_vect.transform(X_en_train)
Val_X_Tfidf = Tfidf_vect.transform(X_en_val)

In [20]:
X_pl_train = df_pl_train['Text']
y_pl_train = df_pl_train['Category']
X_pl_val = df_pl_val['Text']
y_pl_val = df_pl_val['Category']
X_pl_train_tokens= [str(word_tokenize(entry)) for entry in X_pl_train]
Tfidf_vect_pl = TfidfVectorizer(max_features=5000)
Tfidf_vect_pl.fit(X_pl_train_tokens)
Train_X_Tfidf_pl = Tfidf_vect_pl.transform(X_pl_train)
Val_X_Tfidf_pl = Tfidf_vect_pl.transform(X_pl_val)

In [15]:
def training_pipeline_svm(Train_X_Tfidf, Val_X_Tfidf, y_train, y_val):
    SVM = svm.SVC(C=1.0, kernel='linear', degree=3, gamma='auto')
    SVM.fit(Train_X_Tfidf,y_train)
    predictions_SVM = SVM.predict(Val_X_Tfidf)
    accuracy = accuracy_score(predictions_SVM, y_val)
    f1 = f1_score(predictions_SVM, y_val, average='macro')
    precision = precision_score(predictions_SVM, y_val, average='macro')
    recall = recall_score(predictions_SVM, y_val, average='macro')
    confusion_mat = confusion_matrix(predictions_SVM, y_val)
    confusion_df = pd.DataFrame(confusion_mat, columns=np.unique(y_val), index=np.unique(y_val))
    print("SVM Accuracy Score:", accuracy * 100)
    print("SVM:", f1 * 100)
    print("SVM Precision:", precision * 100)
    print("SVM Recall:", recall * 100)
    print("Confusion Matrix:")
    print(confusion_df)
    return predictions_SVM

In [16]:
training_pipeline_svm(Train_X_Tfidf, Val_X_Tfidf, y_train, y_val)

SVM Accuracy Score: 66.90929451287793
SVM: 67.33923910829989
SVM Precision: 66.93551315588063
SVM Recall: 69.67641489974432
Confusion Matrix:
    0   1    2   3   4   5   6   7   8   9   12  13   14   15  16  17
0   77   8    2   1   0   9   1   5   2   4   5   3    0    0   4   3
1    1  63    2   1   2   2  11   0   2   2   8   3    0    2   2   4
2    0   1  100   0   2   1   0   2   4   1   2   0    0    0   1   0
3   17   8    2  92   3  21   4   6   3   6  18   5    2    3   9   4
4    4   0    3   1  52   1   4   6   4   0   0   2    0    0   3   2
5   12   5    0   1   5  75   5   7   3   8   4  10    0    2   9   9
6    1   2    0   1   2   3  50   4   2   1   2   1    0    0   2   1
7    0   0    1   0   1   1   2  65   8   0   3   3    0    0   1   0
8    2   3    5   0  13   0   0  17  66   0   2   4    0    0   8   2
9    6   3    5   0   0   1   0   3   0  70   6   3    1    6   2   0
12   5   8    2   5   4   3   3   3   3   7  60   5    1    4   7   2
13   0   0    2   

array([ 1,  4,  4, ...,  3, 16,  2])

In [21]:
training_pipeline_svm(Train_X_Tfidf_pl, Val_X_Tfidf_pl, y_pl_train, y_pl_val)

SVM Accuracy Score: 98.15789473684211
SVM: 64.63560334528077
SVM Precision: 62.03703703703704
SVM Recall: 71.8694885361552
Confusion Matrix:
    3   5   6   10  11  12  18  19  20
3    0   2   2   0   0   0   0   0   0
5    0   1   0   0   0   0   0   0   0
6    0   0   1   0   0   1   0   0   0
10   0   0   0  89   0   0   0   0   0
11   0   0   0   0  60   0   0   0   0
12   0   0   0   0   0   0   0   0   0
18   1   0   1   0   0   0  61   0   0
19   0   0   0   0   0   0   0  76   0
20   0   0   0   0   0   0   0   0  85


C:\Users\Janek\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


array([20, 10, 11, 11, 20, 18, 10, 10, 20, 18, 19, 19, 10, 10, 10, 19, 18,
       20, 10, 20, 20, 10, 19, 11, 19, 11, 10, 19, 20, 18, 20, 19, 19, 20,
       19, 20, 18, 18, 11, 10, 18, 18, 11, 20, 19, 20, 10, 20, 10, 20, 20,
       11, 18, 18, 11, 18, 11, 18, 10, 20, 19, 18, 10, 20, 10, 11, 19, 11,
       20, 19, 20, 20, 10, 10, 19, 10, 11, 19,  3, 20, 18, 10, 10, 20, 10,
       19, 18, 19, 11, 11, 11, 18, 19, 20, 19, 11, 11, 18, 19, 18, 19, 20,
       20, 19, 10, 20, 10, 18, 20,  6, 10, 20, 19, 19, 18, 11, 11, 19, 11,
       10, 20, 18, 19, 19, 19, 20, 11, 10, 18, 10, 11, 20, 19, 19, 11, 20,
       19, 18, 10, 19, 18, 19, 19, 10, 11, 11, 19, 10, 19, 10, 19, 10, 11,
       20, 18, 10, 10, 11, 20, 19, 18, 11, 11, 18, 20, 11, 20, 19, 20, 20,
       10, 20, 18, 11, 18, 11, 10, 19, 18, 10, 20, 11, 20, 18, 10, 11, 19,
       10, 11, 20, 19, 11, 19, 18, 20,  3, 19, 20, 18, 10, 20, 10, 10, 19,
       11, 10, 18, 19, 19, 20, 18, 20, 20, 20, 20, 19, 10, 19, 20, 18, 18,
       19, 10, 19, 11, 10

## Naive Bayes

In [33]:
def training_pipeline_bayes(Train_X_Tfidf, Val_X_Tfidf, y_train, y_val):
    Naive = naive_bayes.MultinomialNB()
    Naive.fit(Train_X_Tfidf, y_train)
    predictions_NB = Naive.predict(Val_X_Tfidf)
    accuracy = accuracy_score(predictions_NB, y_val)
    f1 = f1_score(predictions_NB, y_val, average='macro')
    precision = precision_score(predictions_NB, y_val, average='macro')
    recall = recall_score(predictions_NB, y_val, average='macro')
    print("Naive Bayes Accuracy Score:", accuracy * 100)
    print("Naive Bayes F1 Score:", f1 * 100)
    print("Naive Bayes Precision:", precision * 100)
    print("Naive Bayes Recall:", recall * 100)
    return predictions_NB

In [34]:
training_pipeline_bayes(Train_X_Tfidf, Val_X_Tfidf, y_en_train, y_en_val)

Naive Bayes Accuracy Score: 61.03023516237403
Naive Bayes F1 Score: 59.9368183149972
Naive Bayes Precision: 60.98390565838439
Naive Bayes Recall: 67.11620324065647


array([ 1,  4, 16, ...,  5, 16,  2])

In [35]:
training_pipeline_bayes(Train_X_Tfidf_pl, Val_X_Tfidf_pl, y_pl_train, y_pl_val)

Naive Bayes Accuracy Score: 97.63157894736842
Naive Bayes F1 Score: 54.91595719271177
Naive Bayes Precision: 55.55555555555556
Naive Bayes Recall: 54.30045699213968


C:\Users\Janek\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


array([20, 10, 11, 11, 20, 18, 10, 10, 20, 18, 19, 19, 10, 10, 10, 19, 18,
       20, 10, 20, 20, 10, 19, 11, 19, 11, 10, 19, 20, 18, 20, 19, 19, 20,
       19, 20, 18, 18, 11, 10, 18, 18, 11, 20, 19, 20, 10, 20, 10, 20, 20,
       11, 18, 18, 11, 18, 11, 18, 10, 20, 19, 18, 10, 20, 10, 11, 19, 11,
       20, 19, 20, 20, 10, 10, 19, 10, 11, 19, 20, 20, 18, 10, 10, 20, 10,
       19, 18, 19, 11, 11, 11, 18, 19, 20, 19, 11, 11, 18, 19, 18, 19, 20,
       20, 19, 10, 20, 10, 18, 20, 10, 10, 20, 19, 19, 18, 11, 11, 19, 11,
       10, 20, 18, 19, 19, 19, 20, 11, 10, 18, 10, 11, 20, 19, 19, 11, 20,
       19, 18, 10, 19, 18, 19, 19, 10, 11, 11, 19, 10, 19, 10, 19, 10, 11,
       20, 18, 10, 10, 11, 20, 19, 18, 11, 11, 18, 20, 11, 20, 19, 20, 20,
       10, 20, 18, 11, 18, 11, 10, 19, 18, 10, 20, 11, 20, 18, 10, 11, 19,
       10, 11, 20, 19, 11, 19, 18, 20, 10, 19, 20, 18, 10, 20, 10, 10, 19,
       11, 10, 18, 19, 19, 20, 18, 20, 20, 20, 20, 19, 10, 19, 20, 18, 18,
       19, 10, 19, 11, 10

## Logistic Regression

In [27]:
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer

def training_pipeline_lr(Train_X_Tfidf, Val_X_Tfidf, y_train, y_val):
    LR = LogisticRegression()
    LR.fit(Train_X_Tfidf, y_train)
    predictions_LR = LR.predict(Val_X_Tfidf)
    accuracy = accuracy_score(predictions_LR, y_val)
    f1 = f1_score(predictions_LR, y_val, average='macro')
    precision = precision_score(predictions_LR, y_val, average='macro')
    recall = recall_score(predictions_LR, y_val, average='macro')
    confusion_mat = confusion_matrix(predictions_LR, y_val)
    
    # Convert confusion matrix to DataFrame for better visualization
    confusion_df = pd.DataFrame(confusion_mat, columns=np.unique(y_val), index=np.unique(y_val))
    
    print("Logistic Regression Accuracy Score:", accuracy * 100)
    print("Logistic Regression F1 Score:", f1 * 100)
    print("Logistic Regression Precision:", precision * 100)
    print("Logistic Regression Recall:", recall * 100)
    
    return predictions_LR

In [29]:
training_pipeline_lr(Train_X_Tfidf, Val_X_Tfidf, y_en_train, y_en_val)

Logistic Regression Accuracy Score: 66.90929451287793
Logistic Regression F1 Score: 67.3617525714085
Logistic Regression Precision: 67.07964365927054
Logistic Regression Recall: 69.73853738583497


array([ 1,  4,  4, ...,  3, 16,  2])

In [30]:
training_pipeline_lr(Train_X_Tfidf_pl, Val_X_Tfidf_pl, y_pl_train, y_pl_val)

Logistic Regression Accuracy Score: 97.63157894736842
Logistic Regression F1 Score: 54.871794871794876
Logistic Regression Precision: 55.55555555555556
Logistic Regression Recall: 54.267310789049915


C:\Users\Janek\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


array([20, 10, 11, 11, 20, 18, 10, 10, 20, 18, 19, 19, 10, 10, 10, 19, 18,
       20, 10, 20, 20, 10, 19, 11, 19, 11, 10, 19, 20, 18, 20, 19, 19, 20,
       19, 20, 18, 18, 11, 10, 18, 18, 11, 20, 19, 20, 10, 20, 10, 20, 20,
       11, 18, 18, 11, 18, 11, 18, 10, 20, 19, 18, 10, 20, 10, 11, 19, 11,
       20, 19, 20, 20, 10, 10, 19, 10, 11, 19, 18, 20, 18, 10, 10, 20, 10,
       19, 18, 19, 11, 11, 11, 18, 19, 20, 19, 11, 11, 18, 19, 18, 19, 20,
       20, 19, 10, 20, 10, 18, 20,  6, 10, 20, 19, 19, 18, 11, 11, 19, 11,
       10, 20, 18, 19, 19, 19, 20, 11, 10, 18, 10, 11, 20, 19, 19, 11, 20,
       19, 18, 10, 19, 18, 19, 19, 10, 11, 11, 19, 10, 19, 10, 19, 10, 11,
       20, 18, 10, 10, 11, 20, 19, 18, 11, 11, 18, 20, 11, 20, 19, 20, 20,
       10, 20, 18, 11, 18, 11, 10, 19, 18, 10, 20, 11, 20, 18, 10, 11, 19,
       10, 11, 20, 19, 11, 19, 18, 20, 18, 19, 20, 18, 10, 20, 10, 10, 19,
       11, 10, 18, 19, 19, 20, 18, 20, 20, 20, 20, 19, 10, 19, 20, 18, 18,
       19, 10, 19, 11, 10

In [14]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import model_selection, naive_bayes, svm
from sklearn.metrics import accuracy_score
from nltk.tokenize import word_tokenize

In [22]:
X_train = df['text_col']
y_train = df['target']

In [56]:
X_train_tokens= [str(word_tokenize(entry)) for entry in X_train]

In [58]:
df['tokens']= (X_train_tokens)

In [73]:
df['tokens']

0    ['To', 'jest', 'przykładowe', 'zdanie', 'po', ...
1    ['This', 'is', 'an', 'example', 'sentence', 'i...
2    ['Łatwość', 'korzystania', 'z', 'tego', 'narzę...
3    ['The', 'ease', 'of', 'use', 'of', 'this', 'to...
4            ['Cześć', '!', 'Jak', 'się', 'masz', '?']
5             ['Hello', '!', 'How', 'are', 'you', '?']
Name: tokens, dtype: object

In [59]:
df

,text_col,target,final,token,tokens
0,To jest przykładowe zdanie po polsku.,1,"['Hello', '!', 'How', 'are', 'you', '?']","[To, jest, przykładowe, zdanie, po, polsku, .]","['To', 'jest', 'przykładowe', 'zdanie', 'po', ..."
1,This is an example sentence in English.,2,"['Hello', '!', 'How', 'are', 'you', '?']","[This, is, an, example, sentence, in, English, .]","['This', 'is', 'an', 'example', 'sentence', 'i..."
2,Łatwość korzystania z tego narzędzia jest nies...,3,"['Hello', '!', 'How', 'are', 'you', '?']","[Łatwość, korzystania, z, tego, narzędzia, jes...","['Łatwość', 'korzystania', 'z', 'tego', 'narzę..."
3,The ease of use of this tool is amazing.,4,"['Hello', '!', 'How', 'are', 'you', '?']","[The, ease, of, use, of, this, tool, is, amazi...","['The', 'ease', 'of', 'use', 'of', 'this', 'to..."
4,Cześć! Jak się masz?,5,"['Hello', '!', 'How', 'are', 'you', '?']","[Cześć, !, Jak, się, masz, ?]","['Cześć', '!', 'Jak', 'się', 'masz', '?']"
5,Hello! How are you?,6,"['Hello', '!', 'How', 'are', 'you', '?']","[Hello, !, How, are, you, ?]","['Hello', '!', 'How', 'are', 'you', '?']"


In [60]:
Tfidf_vect = TfidfVectorizer(max_features=5000)
Tfidf_vect.fit(X_train_tokens)
Train_X_Tfidf = Tfidf_vect.transform(X_train)


In [45]:
Train_X_Tfidf

<6x4 sparse matrix of type '<class 'numpy.float64'>'
	with 4 stored elements in Compressed Sparse Row format>

In [62]:
print(Tfidf_vect.vocabulary_)

{'to': 26, 'jest': 12, 'przykładowe': 20, 'zdanie': 30, 'po': 18, 'polsku': 19, 'this': 25, 'is': 10, 'an': 1, 'example': 6, 'sentence': 21, 'in': 9, 'english': 5, 'łatwość': 31, 'korzystania': 13, 'tego': 23, 'narzędzia': 15, 'niesamowita': 16, 'the': 24, 'ease': 4, 'of': 17, 'use': 28, 'tool': 27, 'amazing': 0, 'cześć': 3, 'jak': 11, 'się': 22, 'masz': 14, 'hello': 7, 'how': 8, 'are': 2, 'you': 29}


In [63]:
print(Train_X_Tfidf)

  (0, 30)	0.41987080495717866
  (0, 26)	0.41987080495717866
  (0, 20)	0.41987080495717866
  (0, 19)	0.41987080495717866
  (0, 18)	0.41987080495717866
  (0, 12)	0.3443000664000145
  (1, 25)	0.32554486881123057
  (1, 21)	0.39699901178248725
  (1, 10)	0.32554486881123057
  (1, 9)	0.39699901178248725
  (1, 6)	0.39699901178248725
  (1, 5)	0.39699901178248725
  (1, 1)	0.39699901178248725
  (2, 31)	0.41987080495717866
  (2, 23)	0.41987080495717866
  (2, 16)	0.41987080495717866
  (2, 15)	0.41987080495717866
  (2, 13)	0.41987080495717866
  (2, 12)	0.3443000664000145
  (3, 28)	0.3109123445439382
  (3, 27)	0.3109123445439382
  (3, 25)	0.25495257018877393
  (3, 24)	0.3109123445439382
  (3, 17)	0.6218246890878764
  (3, 10)	0.25495257018877393
  (3, 4)	0.3109123445439382
  (3, 0)	0.3109123445439382
  (4, 22)	0.5
  (4, 14)	0.5
  (4, 11)	0.5
  (4, 3)	0.5
  (5, 29)	0.5
  (5, 8)	0.5
  (5, 7)	0.5
  (5, 2)	0.5


In [64]:
Naive = naive_bayes.MultinomialNB()
Naive.fit(Train_X_Tfidf,y_train)

MultinomialNB()

In [76]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import model_selection, naive_bayes, svm
from sklearn.metrics import accuracy_score
from nltk.tokenize import word_tokenize

In [77]:
def training_pipeline(df_train, df_test):
    X_train = df_train['text_col']
    y_train = df_test['target']
    X_test = df_train['text_col']
    y_test = df_test['target']
    X_train_tokens= [str(word_tokenize(entry)) for entry in X_train]
    Tfidf_vect = TfidfVectorizer(max_features=5000)
    Tfidf_vect.fit(X_train_tokens)
    Train_X_Tfidf = Tfidf_vect.transform(X_train)
    Test_X_Tfidf = Tfidf_vect.transform(Test_X)
    
    Naive = naive_bayes.MultinomialNB()
    Naive.fit(Train_X_Tfidf, y_train)
    predictions_NB = Naive.predict(Test_X_Tfidf)
    print("Naive Bayes Accuracy Score -> ",accuracy_score(predictions_NB, Test_Y)*100)
    return predictions_NB

In [75]:
training_pipeline(df)

MultinomialNB()